In [8]:
import pandas as pd
import numpy as np
import sqlite3

## Config

In [4]:
# grid configuration over all times
depth_levels = [0, 50, 100, 200, 300, 400, 500, 1000, 1500, 2000, 3000, 4000, 5000]
grid_config = {"param_tables": ["P_TEMPERATURE", "P_SALINITY", "P_OXYGEN", "P_NITRATE", "P_SILICATE", "P_PHOSPHATE"],
               "lat_min": 0,
               "lat_max": 70,
               "dlat": 1,
               "lon_min": -77,
               "lon_max": 30,
               "dlon": 1,
               "z_min": None,
               "z_max": None,
               "dz": None,
               "z_array": np.array(depth_levels),
               "time_min": "1772-01-01 00:00:00",
               "time_max": "2020-07-08 04:45:00",
               "mode": "Y",
               "selection": None,
               "dtime": 300,
               "note": "northern Atlantic, all times, larger depth steps, more params, exclude Pacific"
             }

param_tables = grid_config["param_tables"]
lat_min = grid_config["lat_min"]
lat_max = grid_config["lat_max"]
dlat = grid_config["dlat"]
lon_min = grid_config["lon_min"]
lon_max = grid_config["lon_max"]
dlon = grid_config["dlon"]
z_min = grid_config["z_min"]
z_max = grid_config["z_max"]
dz = grid_config["dz"]
z_array = np.array(grid_config["z_array"])
time_min = grid_config["time_min"]
time_max = grid_config["time_max"]
mode = grid_config["mode"]
selection = grid_config["selection"]
dtime = grid_config["dtime"]

In [9]:
# database connection
db_path = "../missing_value_imputation/data/6_comfort_potT.sqlite"
conn = sqlite3.connect(db_path)

In [11]:
# define bathymetry parameters
bathymetry_grid_path = "../missing_value_imputation/data/bathymetry/gebco_2022_sub_ice_topo/gebco_2022_sub_ice_topo.nc"
lat_variable = "lat"
lon_variable = "lon"
depth_variable = "elevation"

## Get spatially mapped data

In [12]:
include_z_max = True
z_eq = "<=" if include_z_max else "<"
z_min = min(z_array)
z_max = max(z_array)

def load_param_table(connection, table):
    q = f"select LATITUDE, LONGITUDE, LEV_M, DATEANDTIME, VAL " \
    f"from {table} where " \
    f"LATITUDE >= {lat_min} and LATITUDE <= {lat_max} and " \
    f"LONGITUDE >= {lon_min} and LONGITUDE <= {lon_max} and " \
    f"LEV_M >= {z_min} and LEV_M {z_eq} {z_max} and " \
    f"DATEANDTIME >= '{time_min}' and DATEANDTIME <= '{time_max}' " \
    f";"
    print(q)   
    cur = connection.execute(q)
    temp = pd.DataFrame(cur.fetchall(), columns=[x[0] for x in cur.description])

    temp["DATEANDTIME"] = pd.to_datetime(temp["DATEANDTIME"]).dt.strftime("%Y-%m-%d %H:%M:%S")
    return temp

In [14]:
# load data tables from database
nitrate = load_param_table(conn, "P_NITRATE")
silicate = load_param_table(conn, "P_SILICATE")
phosphate = load_param_table(conn, "P_PHOSPHATE")
temperature = load_param_table(conn, "P_TEMPERATURE")
salinity = load_param_table(conn, "P_SALINITY")
oxygen = load_param_table(conn, "P_OXYGEN")

select LATITUDE, LONGITUDE, LEV_M, DATEANDTIME, VAL from P_NITRATE where LATITUDE >= 0 and LATITUDE <= 70 and LONGITUDE >= -77 and LONGITUDE <= 30 and LEV_M >= 0 and LEV_M <= 5000 and DATEANDTIME >= '1772-01-01 00:00:00' and DATEANDTIME <= '2020-07-08 04:45:00' ;
select LATITUDE, LONGITUDE, LEV_M, DATEANDTIME, VAL from P_SILICATE where LATITUDE >= 0 and LATITUDE <= 70 and LONGITUDE >= -77 and LONGITUDE <= 30 and LEV_M >= 0 and LEV_M <= 5000 and DATEANDTIME >= '1772-01-01 00:00:00' and DATEANDTIME <= '2020-07-08 04:45:00' ;
select LATITUDE, LONGITUDE, LEV_M, DATEANDTIME, VAL from P_PHOSPHATE where LATITUDE >= 0 and LATITUDE <= 70 and LONGITUDE >= -77 and LONGITUDE <= 30 and LEV_M >= 0 and LEV_M <= 5000 and DATEANDTIME >= '1772-01-01 00:00:00' and DATEANDTIME <= '2020-07-08 04:45:00' ;
select LATITUDE, LONGITUDE, LEV_M, DATEANDTIME, VAL from P_TEMPERATURE where LATITUDE >= 0 and LATITUDE <= 70 and LONGITUDE >= -77 and LONGITUDE <= 30 and LEV_M >= 0 and LEV_M <= 5000 and DATEANDTIME >= '1

In [17]:
# store the filtered data
nitrate.to_csv("data/P_NITRATE_filtered.csv", index=False)
silicate.to_csv("data/P_SILICATE_filtered.csv", index=False)
phosphate.to_csv("data/P_PHOSPHATE_filtered.csv", index=False)
temperature.to_csv("data/P_TEMPERATURE_filtered.csv", index=False)
salinity.to_csv("data/P_SALINITY_filtered.csv", index=False)
oxygen.to_csv("data/P_OXYGEN_filtered.csv", index=False)

In [ ]:
# load data tables from csv files
nitrate = pd.read_csv("data/P_NITRATE_filtered.csv")
silicate = pd.read_csv("data/P_SILICATE_filtered.csv")
phosphate = pd.read_csv("data/P_PHOSPHATE_filtered.csv")
temperature = pd.read_csv("data/P_TEMPERATURE_filtered.csv")
salinity = pd.read_csv("data/P_SALINITY_filtered.csv")
oxygen = pd.read_csv("data/P_OXYGEN_filtered.csv")

## Examine temporal data density

In [19]:
time_bins = pd.date_range(time_min, time_max, freq="D")

def plot_monthly_avg_over_time(tables, plot_titles, measure="mean", d_or_m="m"):  
    aggregated_tables = []
    for table in tables:
        # drop nan values
        table = table.dropna()

        # drop land values
        table = table.drop(table[~table["water"]].index)
        
        # binning
        table["DATEANDTIME_dbinned"] = pd.cut(pd.to_datetime(table["DATEANDTIME"], format="%Y-%m-%d %H:%M:%S"), bins=time_bins, right=False, labels=time_bins[:-1])
        table["DATEANDTIME_mbinned"] = table["DATEANDTIME_dbinned"].dt.month
        table["DATEANDTIME_ybinned"] = table["DATEANDTIME_dbinned"].dt.year.astype(int)

        # binning and averaging
        if d_or_m == "d": 
            col = "DATEANDTIME_dbinned"
            agg_temp = table.drop(["DATEANDTIME", "DATEANDTIME_mbinned", "DATEANDTIME_ybinned"], axis=1).groupby(["DATEANDTIME_dbinned"], observed=False).agg(measure).reset_index()  # mean over time
            thresh = agg_temp["VAL"].mean() + agg_temp["VAL"].std()
            agg_temp = agg_temp[agg_temp["VAL"] <= thresh]
            scatter_size = 0.01
            figsize = (20, 1.5*len(tables))
            smoothing=24
        elif d_or_m == "m":
            col = "DATEANDTIME_mbinned"
            agg_temp = table.drop(["DATEANDTIME", "DATEANDTIME_dbinned", "DATEANDTIME_ybinned"], axis=1).groupby(["DATEANDTIME_mbinned"], observed=False).agg(measure).reset_index()
            scatter_size = 30
            figsize = (6, 1.2*len(tables))
            smoothing=1
        elif d_or_m == "y":
            col = "DATEANDTIME_ybinned"
            agg_temp = table.drop(["DATEANDTIME", "DATEANDTIME_dbinned", "DATEANDTIME_mbinned"], axis=1).groupby(["DATEANDTIME_ybinned"], observed=False).agg(measure).reset_index()
            agg_temp["DATEANDTIME_ybinned"] = agg_temp["DATEANDTIME_ybinned"].astype(int)
            scatter_size = 10
            figsize = (10, 1.5*len(tables))
            smoothing=1
        aggregated_tables.append(agg_temp)
        
    # plot
    fig, axs = plt.subplots(ncols=1, nrows=len(tables), sharex=True, figsize=figsize)
    
    for i in range(len(tables)):
        t = aggregated_tables[i]
        if d_or_m == "m":  # or measure == "count": 
            t[col] = pd.to_datetime(t[col])
            #sns.lineplot(data=t,x= col, y="VAL", ax=axs[i])
            axs[i].plot(col, "VAL", data=t)
        t["smooth"] = gaussian_filter1d(t["VAL"], sigma=smoothing)
        axs[i].plot(col, "smooth", data=t, c="black")
        axs[i].scatter(col, "VAL", data=t, s=scatter_size)
        axs[i].set_title(plot_titles[i])
    
    plt.ylabel(measure.capitalize())
    plt.xlabel("Time")
    plt.tight_layout()
    plt.show()

### Monthly

In [20]:
plot_monthly_avg_over_time([temperature, salinity, oxygen, nitrate, silicate, phosphate], measure="mean", d_or_m="m", 
                           plot_titles=["Temperature [°C]", "Salinity [psu]", "Oxygen [µmol·kg-1]", 
                                        "Nitrate [µmol·kg-1]", "Silicate [µmol·kg-1]", "Phosphate [µmol·kg-1]"])

KeyError: 'water'

In [ ]:
plot_monthly_avg_over_time([temperature, salinity, oxygen, nitrate, silicate, phosphate], measure="count",
                           plot_titles=["Temperature [°C]", "Salinity [psu]", "Oxygen [µmol·kg-1]", 
                                        "Nitrate [µmol·kg-1]", "Silicate [µmol·kg-1]", "Phosphate [µmol·kg-1]"])

### Daily

In [ ]:
plot_monthly_avg_over_time([temperature, salinity, oxygen, nitrate, silicate, phosphate], measure="mean", d_or_m="d", 
                           plot_titles=["Temperature [°C]", "Salinity [psu]", "Oxygen [µmol·kg-1]", 
                                        "Nitrate [µmol·kg-1]", "Silicate [µmol·kg-1]", "Phosphate [µmol·kg-1]"])

In [ ]:
plot_monthly_avg_over_time([temperature, salinity, oxygen, nitrate, silicate, phosphate], measure="count", d_or_m="d", 
                           plot_titles=["Temperature [°C]", "Salinity [psu]", "Oxygen [µmol·kg-1]", 
                                        "Nitrate [µmol·kg-1]", "Silicate [µmol·kg-1]", "Phosphate [µmol·kg-1]"])

### Yearly

In [ ]:
plot_monthly_avg_over_time([temperature, salinity, oxygen, nitrate, silicate, phosphate], measure="mean", d_or_m="y", 
                           plot_titles=["Temperature [°C]", "Salinity [psu]", "Oxygen [µmol·kg-1]", 
                                        "Nitrate [µmol·kg-1]", "Silicate [µmol·kg-1]", "Phosphate [µmol·kg-1]"])

In [ ]:
plot_monthly_avg_over_time([temperature, salinity, oxygen, nitrate, silicate, phosphate], measure="count", d_or_m="y", 
                           plot_titles=["Temperature [°C]", "Salinity [psu]", "Oxygen [µmol·kg-1]", 
                                        "Nitrate [µmol·kg-1]", "Silicate [µmol·kg-1]", "Phosphate [µmol·kg-1]"])

# 1985-1989 analysis